# DATA CLEANING

Adjust column names, handle duplicates and missing values, standardize text fields, merge datasets, and save the new dataframes.

⚠️ **NOTE:** As I said in the previous notebook, in this notebook I share the data cleaning process I conducted. Due to potential future issues and time constraints, I've decided to focus primarily on utilizing only the "books_df" dataset and the one I personally scraped from the Goodreads website, while many other datasets included in this notebook may not be used.

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from IPython.display import Image, display
from IPython.display import HTML

import warnings
warnings.filterwarnings("ignore")

## Load Data

In [2]:
books_df = pd.read_csv('GoodReads_100k_books.csv')
ratings_df = pd.read_csv('Ratings.csv')
books_df_2 = pd.read_csv('Books.csv')
users_df = pd.read_csv('Users.csv')
top100_df = pd.read_csv('Top-100 Trending Books.csv')
customers_df = pd.read_csv('customer reviews.csv')
goodreads_df = pd.read_csv('goodreads_webscrap.csv')

# Attempt to read a CSV file named 'gr_books.csv' into a Pandas DataFrame.
try:
    gr_books_df = pd.read_csv('gr_books.csv', error_bad_lines=False)
    
# If there is an exception (specifically, a ParserError) while parsing the CSV file,
# print an error message that includes the details of the exception.
except pd.errors.ParserError as e:
    print(f"Error parsing 'gr_books.csv': {e}")

Skipping line 3350: expected 12 fields, saw 13
Skipping line 4704: expected 12 fields, saw 13
Skipping line 5879: expected 12 fields, saw 13
Skipping line 8981: expected 12 fields, saw 13



## Check the Data

In [3]:
books_df.head()

,author,bookformat,desc,genre,img,isbn,isbn13,link,pages,rating,reviews,title,totalratings
0,Laurence M. Hauptman,Hardcover,Reveals that several hundred thousand Indians ...,"History,Military History,Civil War,American Hi...",https://i.gr-assets.com/images/S/compressed.ph...,002914180X,9.78E+12,https://goodreads.com/book/show/1001053.Betwee...,0,3.52,5,Between Two Fires: American Indians in the Civ...,33
1,"Charlotte Fiell,Emmanuelle Dirix",Paperback,Fashion Sourcebook - 1920s is the first book i...,"Couture,Fashion,Historical,Art,Nonfiction",https://i.gr-assets.com/images/S/compressed.ph...,1906863482,9.78E+12,https://goodreads.com/book/show/10010552-fashi...,576,4.51,6,Fashion Sourcebook 1920s,41
2,Andy Anderson,Paperback,The seminal history and analysis of the Hungar...,"Politics,History",https://i.gr-assets.com/images/S/compressed.ph...,948984147,9.78E+12,https://goodreads.com/book/show/1001077.Hungar...,124,4.15,2,Hungary 56,26
3,Carlotta R. Anderson,Hardcover,"""All-American Anarchist"" chronicles the life a...","Labor,History",https://i.gr-assets.com/images/S/compressed.ph...,814327079,9.78E+12,https://goodreads.com/book/show/1001079.All_Am...,324,3.83,1,All-American Anarchist: Joseph A. Labadie and ...,6
4,Jean Leveille,NaN,"Aujourdâ€™hui, lâ€™oiseau nous invite Ã sa ta...",NaN,https://i.gr-assets.com/images/S/compressed.ph...,2761920813,NaN,https://goodreads.com/book/show/10010880-les-o...,177,4.00,1,Les oiseaux gourmands,1


In [4]:
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
books_df_2.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [6]:
users_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [7]:
top100_df.head()

,Rank,book title,book price,rating,author,year of publication,genre,url
0,1,"Iron Flame (The Empyrean, 2)",18.42,4.1,Rebecca Yarros,2023,Fantasy Romance,amazon.com/Iron-Flame-Empyrean-Rebecca-Yarros/...
1,2,The Woman in Me,20.93,4.5,Britney Spears,2023,Memoir,amazon.com/Woman-Me-Britney-Spears/dp/16680090...
2,3,My Name Is Barbra,31.50,4.5,Barbra Streisand,2023,Autobiography,amazon.com/My-Name-Barbra-Streisand/dp/0525429...
3,4,"Friends, Lovers, and the Big Terrible Thing: A...",23.99,4.4,Matthew Perry,2023,Memoir,amazon.com/Friends-Lovers-Big-Terrible-Thing/d...
4,5,How to Catch a Turkey,5.65,4.8,Adam Wallace,2018,"Childrens, Fiction",amazon.com/How-Catch-Turkey-Adam-Wallace/dp/14...


In [8]:
customers_df.head()

,Sno,book name,review title,reviewer,reviewer rating,review description,is_verified,date,timestamp,ASIN
0,0,The Woman in Me,Unbelievably impressive. Her torn life on paper.,Murderess Marbie,4,I'm only a third way in. Shipped lightening fa...,True,26-10-2023,"Reviewed in the United States October 26, 2023",1668009048
1,1,The Woman in Me,What a heartbreaking story,L J,5,"""There have been so many times when I was scar...",True,06-11-2023,"Reviewed in the United States November 6, 2023",1668009048
2,2,The Woman in Me,Britney you are so invincible! You are an insp...,Jamie,5,The media could not be loaded. I personally ha...,True,01-11-2023,"Reviewed in the United States November 1, 2023",1668009048
3,3,The Woman in Me,"Fast Read, Sad Story",KMG,5,I have been a fan of Britney's music since the...,True,25-10-2023,"Reviewed in the United States October 25, 2023",1668009048
4,4,The Woman in Me,"Buy it, it’s worth the read!",Stephanie Brown,5,"Whether or not you’re a fan, it’s a great read...",True,01-11-2023,"Reviewed in the United States November 1, 2023",1668009048


In [9]:
gr_books_df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


In [10]:
goodreads_df.head()

,Title,Author,Image URL,Rating
0,"Fourth Wing (The Empyrean, #1)",Rebecca Yarros,https://i.gr-assets.com/images/S/compressed.ph...,"4.63 avg rating — 844,282 ratings"
1,Happy Place,Emily Henry,https://i.gr-assets.com/images/S/compressed.ph...,"4.06 avg rating — 574,386 ratings"
2,Yellowface,R.F. Kuang,https://i.gr-assets.com/images/S/compressed.ph...,"3.87 avg rating — 227,094 ratings"
3,"Love, Theoretically",Ali Hazelwood,https://i.gr-assets.com/images/S/compressed.ph...,"4.17 avg rating — 239,576 ratings"
4,"Divine Rivals (Letters of Enchantment, #1)",Rebecca Ross,https://i.gr-assets.com/images/S/compressed.ph...,"4.26 avg rating — 161,237 ratings"


## Standardize column names to snake_case

In [11]:
def to_snake_case(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
    return df

In [12]:
# Apply to books_df
books_df = to_snake_case(books_df)

# Apply to ratings_df
ratings_df = to_snake_case(ratings_df)

# Apply to books_df_2
books_df_2 = to_snake_case(books_df_2)

# Apply to users_df
users_df = to_snake_case(users_df)

# Apply to top100_df
top100_df = to_snake_case(top100_df)

# Apply to customers_df
customers_df = to_snake_case(customers_df)

# Apply to goodreads_df
goodreads_df = to_snake_case(goodreads_df)

# Apply to gr_books_df
gr_books_df = to_snake_case(gr_books_df)

In [13]:
# Check column names
books_df.columns

Index(['author', 'bookformat', 'desc', 'genre', 'img', 'isbn', 'isbn13',
       'link', 'pages', 'rating', 'reviews', 'title', 'totalratings'],
      dtype='object')

I will change:

- 'bookformat' to book_format
- 'desc' to description
- 'img' to image_url
- 'link' to url
- 'totalratings' to total_ratings

In [14]:
# Rename columns in books_df
books_df.rename(columns={
    'bookformat': 'book_format',
    'desc': 'description',
    'img': 'image_url',
    'link': 'url',
    'totalratings': 'total_ratings'
}, inplace=True)

# Check
books_df.columns

Index(['author', 'book_format', 'description', 'genre', 'image_url', 'isbn',
       'isbn13', 'url', 'pages', 'rating', 'reviews', 'title',
       'total_ratings'],
      dtype='object')

In [15]:
ratings_df.columns

Index(['user_id', 'isbn', 'book_rating'], dtype='object')

I will change:

- 'book_rating' to rating

In [16]:
# Rename the 'book_rating' column to 'rating' in ratings_df
ratings_df.rename(columns={'book_rating': 'rating'}, inplace=True)

ratings_df.columns

Index(['user_id', 'isbn', 'rating'], dtype='object')

In [17]:
books_df_2.columns

Index(['isbn', 'book_title', 'book_author', 'year_of_publication', 'publisher',
       'image_url_s', 'image_url_m', 'image_url_l'],
      dtype='object')

I will change:

- 'book_title' to title
- 'book_author' to author
- 'year_of_publication' to year

In [18]:
# Rename columns in books_df_2
books_df_2.rename(columns={'book_title': 'title', 'book_author': 'author', 'year_of_publication': 'year'}, inplace=True)

books_df_2.columns

Index(['isbn', 'title', 'author', 'year', 'publisher', 'image_url_s',
       'image_url_m', 'image_url_l'],
      dtype='object')

In [19]:
users_df.columns

Index(['user_id', 'location', 'age'], dtype='object')

In [20]:
top100_df.columns

Index(['rank', 'book_title', 'book_price', 'rating', 'author',
       'year_of_publication', 'genre', 'url'],
      dtype='object')

I will change:

- 'book_title' to title
- 'book_price' to price
- 'year_of_publication' to year

In [21]:
# Rename columns in top100_df
top100_df.rename(columns={'book_title': 'title', 'book_price': 'price', 'year_of_publication': 'year'}, inplace=True)

top100_df.columns

Index(['rank', 'title', 'price', 'rating', 'author', 'year', 'genre', 'url'], dtype='object')

In [22]:
customers_df.columns

Index(['sno', 'book_name', 'review_title', 'reviewer', 'reviewer_rating',
       'review_description', 'is_verified', 'date', 'timestamp', 'asin'],
      dtype='object')

I will change:

- 'sno' to serial_number
- 'book_name' to title
- 'date' to year

In [23]:
# Rename columns in customers_df
customers_df.rename(columns={'sno': 'serial_number', 'book_name': 'title', 'date': 'year'}, inplace=True)

customers_df.columns

Index(['serial_number', 'title', 'review_title', 'reviewer', 'reviewer_rating',
       'review_description', 'is_verified', 'year', 'timestamp', 'asin'],
      dtype='object')

In [24]:
goodreads_df.columns

Index(['title', 'author', 'image_url', 'rating'], dtype='object')

In [25]:
gr_books_df.columns

Index(['bookid', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', '__num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')

I will change:

- 'bookid' to book_id
- 'authors' to author
- 'average_rating' to rating, 'isbn', 'isbn13'
- 'language_code' to language
- '__num_pages' to pages
- 'text_reviews_count' to reviews_count
- 'publication_date' to year

In [26]:
# Rename columns in gr_books_df
gr_books_df.rename(columns={
    'bookid': 'book_id',
    'authors': 'author',
    'average_rating': 'rating',
    'isbn': 'isbn',
    'isbn13': 'isbn13',
    'language_code': 'language',
    '__num_pages': 'pages',
    'text_reviews_count': 'reviews_count',
    'publication_date': 'year'
}, inplace=True)

gr_books_df.columns

Index(['book_id', 'title', 'author', 'rating', 'isbn', 'isbn13', 'language',
       'pages', 'ratings_count', 'reviews_count', 'year', 'publisher'],
      dtype='object')

## Merge some dataframes on common columns

### Merge books_df_2, ratings_df and users_df

They are from the same dataset.

In [27]:
# Merge users_df and ratings_df on 'user_id'
u_r_df = users_df.merge(ratings_df, on='user_id', how='inner')

# Merge u_r_df and books_df_2 on 'isbn'
users_books_df = u_r_df.merge(books_df_2, on='isbn', how='inner')

In [28]:
users_books_df.head()

,user_id,location,age,isbn,rating,title,author,year,publisher,image_url_s,image_url_m,image_url_l
0,2,"stockton, california, usa",18.0,0195153448,0,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,8,"timmins, ontario, canada",NaN,0002005018,5,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,11400,"ottawa, ontario, canada",49.0,0002005018,0,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
3,11676,"n/a, n/a, n/a",NaN,0002005018,8,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
4,41385,"sudbury, ontario, canada",NaN,0002005018,0,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


### Merge top100_df and customers_df

Both are from the same dataset.

In [29]:
# Merge the DataFrames using the "title" column
best_selling_df = pd.merge(top100_df, customers_df, on='title', how='inner')

best_selling_df.head()

,rank,title,price,rating,author,year_x,genre,url,serial_number,review_title,reviewer,reviewer_rating,review_description,is_verified,year_y,timestamp,asin
0,2,The Woman in Me,20.93,4.5,Britney Spears,2023,Memoir,amazon.com/Woman-Me-Britney-Spears/dp/16680090...,0,Unbelievably impressive. Her torn life on paper.,Murderess Marbie,4,I'm only a third way in. Shipped lightening fa...,True,26-10-2023,"Reviewed in the United States October 26, 2023",1668009048
1,2,The Woman in Me,20.93,4.5,Britney Spears,2023,Memoir,amazon.com/Woman-Me-Britney-Spears/dp/16680090...,1,What a heartbreaking story,L J,5,"""There have been so many times when I was scar...",True,06-11-2023,"Reviewed in the United States November 6, 2023",1668009048
2,2,The Woman in Me,20.93,4.5,Britney Spears,2023,Memoir,amazon.com/Woman-Me-Britney-Spears/dp/16680090...,2,Britney you are so invincible! You are an insp...,Jamie,5,The media could not be loaded. I personally ha...,True,01-11-2023,"Reviewed in the United States November 1, 2023",1668009048
3,2,The Woman in Me,20.93,4.5,Britney Spears,2023,Memoir,amazon.com/Woman-Me-Britney-Spears/dp/16680090...,3,"Fast Read, Sad Story",KMG,5,I have been a fan of Britney's music since the...,True,25-10-2023,"Reviewed in the United States October 25, 2023",1668009048
4,2,The Woman in Me,20.93,4.5,Britney Spears,2023,Memoir,amazon.com/Woman-Me-Britney-Spears/dp/16680090...,4,"Buy it, it’s worth the read!",Stephanie Brown,5,"Whether or not you’re a fan, it’s a great read...",True,01-11-2023,"Reviewed in the United States November 1, 2023",1668009048


### Find common columns between books_df and gr_books_df

This two DataFrames are the ones that I will use for my model. So:

1. **Common Columns**: Identifying common columns is essential for merging, analyzing, or combining data from both DataFrames. It helps ensure that I have consistent information across the two DataFrames. In my case, it helps identify which columns I can use as features for clustering.

2. **Columns Unique to books_df**: These columns are present in `books_df` but not in `gr_books_df`. Knowing these unique columns allows to understand what additional information is available in `books_df`. I may consider using these columns as additional features in my book recommender system.

3. **Columns Unique to gr_books_df**: Similarly, these columns are present in `gr_books_df` but not in `books_df`. Understanding these unique columns helps me to assess what additional data might be available in `gr_books_df`. So I can decide whether to incorporate this information into my recommender system.

By analyzing common and unique columns, I can make informed decisions about which features to include in my book recommender model, ensuring it utilizes all relevant information for effective clustering and recommendations.

In [30]:
# Find common columns between books_df and gr_books_df
common_columns = set(books_df.columns) & set(gr_books_df.columns)

# Find columns unique to books_df
unique_to_books_df = set(books_df.columns) - set(gr_books_df.columns)

# Find columns unique to gr_books_df
unique_to_gr_books_df = set(gr_books_df.columns) - set(books_df.columns)

print("Common Columns:")
print(common_columns)
print("\nColumns Unique to books_df:")
print(unique_to_books_df)
print("\nColumns Unique to gr_books_df:")
print(unique_to_gr_books_df)

Common Columns:
{'rating', 'isbn', 'author', 'isbn13', 'pages', 'title'}

Columns Unique to books_df:
{'reviews', 'description', 'image_url', 'url', 'total_ratings', 'genre', 'book_format'}

Columns Unique to gr_books_df:
{'ratings_count', 'book_id', 'language', 'year', 'publisher', 'reviews_count'}


#### My selected features for clustering are:

1. Description (I will make a sentiment analysis to group them)
2. Genre
3. Pages
4. Rating
5. Year of Publication
6. Language

#### So here's what I should consider doing to gather the missing information for these features:

1. **Description**: The `description` is missing in `gr_books_df`, so I can try to obtain book descriptions from various sources, such as Google Books API, Amazon, or other book-related APIs. I can use the ISBN or book title to fetch descriptions for books missing this information.

2. **Genre**: I can manually categorize books into genres because this information is missing in `gr_books_df`. Alternatively, I can use machine learning techniques, such as natural language processing (NLP), to classify books into genres based on their descriptions or other available information.

3. **Pages**: I have the `pages` information in both dataframes.

4. **Rating**: Ratings are available in both DataFrames, so there's no need to gather additional information for this feature.

5. **Year of Publication**: Missing in `books_df` so I can attempt to fill in missing publication years by searching for the books on platforms like Google Books or Amazon. These platforms often provide publication years for books.

6. **Language**: Missing in `books_df`. The `language` feature may require web scraping or API calls to obtain language information for books. I can use the ISBN or book title to search for language details on book-related websites or language-specific APIs.

In summary, for the missing information, I may need to resort to external sources such as online bookstores, libraries, or book-related APIs to fill in the gaps. Web scraping and data enrichment techniques can be valuable tools in this process, helping me gather comprehensive data for my clustering and recommendation model.

For now I will continue with the data cleaning before filling this missing information and merging both dataframes.

## Null Values

In [31]:
print("Null values in Books:\n" ,books_df.isnull().sum())
print("\nNull values in Users Books:\n" ,users_books_df.isnull().sum())
print("\nNull values in Best Selling:\n" ,best_selling_df.isnull().sum())
print("\nNull values in Goodreads:\n" ,goodreads_df.isnull().sum())
print("\nNull values in Goodreads 2:\n" ,gr_books_df.isnull().sum())

Null values in Books:
 author               0
book_format       3228
description       6772
genre            10467
image_url         3045
isbn             14482
isbn13           11435
url                  0
pages                0
rating               0
reviews              0
title                1
total_ratings        0
dtype: int64

Null values in Users Books:
 user_id             0
location            0
age            277835
isbn                0
rating              0
title               0
author              1
year                0
publisher           2
image_url_s         0
image_url_m         0
image_url_l         4
dtype: int64

Null values in Best Selling:
 rank                  0
title                 0
price                 0
rating                0
author                0
year_x                0
genre                 0
url                   0
serial_number         0
review_title          0
reviewer              0
reviewer_rating       0
review_description    0
is_verified    

This provides information about the presence of null (missing) values in various columns of different dataframes. 

#### Here's a summary of the null values in each dataframe:

**1. Books DataFrame:**
- `book_format`: 3,228 missing values.
- `description`: 6,772 missing values.
- `genre`: 10,467 missing values.
- `image_url`: 3,045 missing values.
- `isbn`: 14,482 missing values.
- `isbn13`: 11,435 missing values.
- `title`: 1 missing value.

**2. Users Books DataFrame:**
- `age`: 277,835 missing values.
- `author`: 1 missing value.
- `publisher`: 2 missing values.
- `image_url_l`: 4 missing values.

**3. Best Selling DataFrame:**
No missing values.

**4. Goodreads DataFrame:**
No missing values.

**5. Goodreads 2 DataFrame:**
No missing values.

This helps me to identify columns with missing values in each dataframe. I may need to handle these missing values through techniques like imputation, removal, or data enrichment.

## Identifying and Removing Unnecessary Columns

#### Books: I will drop `book_format` and `isbn13`.

In [32]:
books_df.columns

Index(['author', 'book_format', 'description', 'genre', 'image_url', 'isbn',
       'isbn13', 'url', 'pages', 'rating', 'reviews', 'title',
       'total_ratings'],
      dtype='object')

In the context of my dataset and the specific objectives of my analysis, I have decided to retain the `isbn` column while removing the `isbn13` one. The reason for this choice is primarily based on the practicality and common usage of the ISBN (International Standard Book Number) as the primary unique identifier for books. The ISBN serves as a widely recognized and effective means of book identification, facilitating searches and categorization. While ISBN13 provides a longer and more detailed identification number, it is not commonly required for my current analytical goals, and the ISBN alone is sufficient for my purposes. By removing the `isbn13` column, I aim to streamline the dataset and reduce redundancy, focusing on the most essential information for my book-related analysis.

In [33]:
books_df.drop(columns=["book_format", 'isbn13'], inplace=True)

books_df.columns

Index(['author', 'description', 'genre', 'image_url', 'isbn', 'url', 'pages',
       'rating', 'reviews', 'title', 'total_ratings'],
      dtype='object')

#### Users Books: I will drop `user_id`, `image_url_s` and `image_url_l`

User_id is unique for each user so I don't need it for my analysis. Also from the images I will just keep `image_url_m` that contains the same information.

In [34]:
users_books_df.columns

Index(['user_id', 'location', 'age', 'isbn', 'rating', 'title', 'author',
       'year', 'publisher', 'image_url_s', 'image_url_m', 'image_url_l'],
      dtype='object')

In [35]:
users_books_df.drop(columns=["user_id", 'image_url_s', 'image_url_l'], inplace=True)

users_books_df.columns

Index(['location', 'age', 'isbn', 'rating', 'title', 'author', 'year',
       'publisher', 'image_url_m'],
      dtype='object')

In [36]:
users_books_df.rename(columns={'image_url_m': 'image_url'}, inplace=True)

users_books_df.columns

Index(['location', 'age', 'isbn', 'rating', 'title', 'author', 'year',
       'publisher', 'image_url'],
      dtype='object')

**Best Selling: I will drop `rank`, `serial_number`, `review_title`, `reviewer`, `reviewer_rating`, `is_verified`, `year_y`, `review_description`, `timestamp` and `asin`**

In [37]:
best_selling_df.columns

Index(['rank', 'title', 'price', 'rating', 'author', 'year_x', 'genre', 'url',
       'serial_number', 'review_title', 'reviewer', 'reviewer_rating',
       'review_description', 'is_verified', 'year_y', 'timestamp', 'asin'],
      dtype='object')

I will drop them because:

- `serial number`: I don't need this one for my analysis.
- `review_title`: Is redundant with the information contained in `review_description`.
- `reviewer`: I don't need the name of each person who has written a review for the book.
- `reviewer_rating`: Is redundant with the average rating in the `rating`column.
- `is_verified`: also don't need this one.
- `year_y`: because it indicates the date when the review was posted and I don't need it for my analysis.
- `rank`, `review_description`, `timestamp` and `asin`: all of them are unique for each book and I don't need them for the recommender.

In [38]:
# List of columns to drop
columns_to_drop = ["rank", "review_description", "timestamp", "asin", 'serial_number', 'review_title', 'reviewer', 'reviewer_rating', 'is_verified', 'year_y']

# Drop columns in the list
best_selling_df.drop(columns=columns_to_drop, inplace=True)

In [39]:
# Rename the 'year_x' column to 'year' 
best_selling_df.rename(columns={'year_x': 'year'}, inplace=True)

best_selling_df.columns

Index(['title', 'price', 'rating', 'author', 'year', 'genre', 'url'], dtype='object')

#### Goodreads: I keep them all

In [40]:
goodreads_df.columns

Index(['title', 'author', 'image_url', 'rating'], dtype='object')

#### Goodreads 2: I will drop `book_id` and `isbn13`

I don't need them since I already have the `isbn` column.

In [41]:
gr_books_df.columns

Index(['book_id', 'title', 'author', 'rating', 'isbn', 'isbn13', 'language',
       'pages', 'ratings_count', 'reviews_count', 'year', 'publisher'],
      dtype='object')

In [42]:
columns_to_drop = ['book_id', 'isbn13']
gr_books_df.drop(columns=columns_to_drop, inplace=True)

gr_books_df.columns

Index(['title', 'author', 'rating', 'isbn', 'language', 'pages',
       'ratings_count', 'reviews_count', 'year', 'publisher'],
      dtype='object')

## Fill missing information using APIs

**Missing info in `books_df`:**

- description: 6,772 missing values.
- genre: 10,467 missing values.
- image_url: 3,045 missing values.
- isbn: 14,482 missing values.
- title: 1 missing value.
- year
- language


**Missing info in `users_books_df`:**

- age: 277,835 missing values.
- author: 1 missing value.
- publisher: 2 missing values.

I will fill the age missing values lately in the EDA part. And since I only have 1 missing value for author and 2 for publisher I will fill them manually checking in the web using the ISBN of the book.


**Missing info in `gr_books_df`:**

- description
- genre

### 1. books_df

I use the Google Books API to fetch missing information based on the 'isbn', 'title', and 'author' from the books_df DataFrame. I will attempt to fetch the missing information using the ISBN. If the ISBN is not available, I will construct a query using both the title and the author.

### Fill missing information using Google Books API


It is a time-consuming process (approximately 4 hours) and doesn't provide comprehensive information for each book, which is why I ultimately choose to skip this step.

But here I share the code that I was trying.

In [43]:
books_df.shape

(100000, 11)

In [44]:
# Create a DataFrame with rows containing null values in any column
null_rows_df = books_df[books_df.isnull().any(axis=1)]
null_rows_df

,author,description,genre,image_url,isbn,url,pages,rating,reviews,title,total_ratings
4,Jean Leveille,"Aujourdâ€™hui, lâ€™oiseau nous invite Ã sa ta...",NaN,https://i.gr-assets.com/images/S/compressed.ph...,2761920813,https://goodreads.com/book/show/10010880-les-o...,177,4.00,1,Les oiseaux gourmands,1
8,Nick Le Neve Walmsley,At the time of her construction in the late 19...,NaN,https://i.gr-assets.com/images/S/compressed.ph...,752456830,https://goodreads.com/book/show/10011431-r101,160,5.00,2,R101: A Pictorial History,3
10,Mark Verman,The earliest medieval Jewish mystical writings...,NaN,https://i.gr-assets.com/images/S/compressed.ph...,791407209,https://goodreads.com/book/show/10011932-the-b...,282,4.75,1,The Books of Contemplation: Medieval Jewish My...,4
27,"Leo Bagrow,Raleigh Ashlin Skelton",This illustrated work is intended to acquaint ...,NaN,NaN,913750336,https://goodreads.com/book/show/10013570-histo...,312,5.00,0,History of Cartography: Enlarged Second Edition,1
30,Luc Milne,NaN,"Erotica,Gay Erotica",https://i.gr-assets.com/images/S/compressed.ph...,943595614,https://goodreads.com/book/show/1001377.The_Mi...,160,3.46,1,The Milk Farm: An Erotic Novel,13
...,...,...,...,...,...,...,...,...,...,...,...
99988,C.T. Khanolkar,A Novella that brings out a story of a person ...,NaN,https://i.gr-assets.com/images/S/compressed.ph...,NaN,https://goodreads.com/book/show/17319699,90,3.14,3,à¤…à¤œà¤—à¤°,14
99989,Claire Farrell,Perditaâ€™s life has gone back to normal in al...,"Fantasy,Paranormal,Young Adult,Fantasy,Shapesh...",https://i.gr-assets.com/images/S/compressed.ph...,NaN,https://goodreads.com/book/show/17319882-purity,0,4.05,64,Purity,1244
99990,Ilja Gort,Ilja Gort is een Nederlandse wijnboer in Frank...,NaN,https://i.gr-assets.com/images/S/compressed.ph...,NaN,https://goodreads.com/book/show/17319892-slurp,246,3.64,1,Slurp!,14
99995,Simon Monk,Design custom printed circuit boards with EAGL...,NaN,https://i.gr-assets.com/images/S/compressed.ph...,71819266,https://goodreads.com/book/show/17320181-make-...,304,4.07,7,Make Your Own PCBs with Eagle: From Schematic ...,56


In [45]:
from concurrent.futures import ThreadPoolExecutor, as_completed

# Function to fetch data from Google Books API
def fetch_book_data(isbn, title, author):
    # Construct the query
    query = isbn if pd.notna(isbn) else f'intitle:{title}+inauthor:{author}'
    url = f"https://www.googleapis.com/books/v1/volumes?q={query}"
    
    # Initialize empty data dictionary
    book_data = {
        'description': None,
        'genres': None,
        'year_of_publication': None,
        'language': None
    }
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
        
        # Parse response if items are present
        if 'items' in response.json():
            book_info = response.json()['items'][0]['volumeInfo']
            
            # Get the necessary information if available
            book_data['description'] = book_info.get('description')
            book_data['genres'] = ', '.join(book_info.get('categories', []))
            book_data['year_of_publication'] = book_info.get('publishedDate', '').split('-')[0]  # Just get the year part
            book_data['language'] = book_info.get('language')
            
    except requests.exceptions.HTTPError as e:
        # Add more error handling here depending on the requirements
        print(f"HTTPError for ISBN/Title '{isbn or title}': {e}")
    
    except requests.exceptions.RequestException as e:
        # Handle any other exceptions
        print(f"RequestException for ISBN/Title '{isbn or title}': {e}")
    
    return book_data

# Function to update the DataFrame
def update_books_df(df):
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_row = {executor.submit(fetch_book_data, row['isbn'], row['title'], row['author']): index for index, row in df.iterrows()}
        
        for future in as_completed(future_to_row):
            index = future_to_row[future]
            try:
                result = future.result()
                # Update the DataFrame if data is found
                if result['description']:
                    df.at[index, 'Description'] = result['description']
                if result['genres']:
                    df.at[index, 'Genre'] = result['genres']
                if result['year_of_publication']:
                    df.at[index, 'YearOfPublication'] = result['year_of_publication']
                if result['language']:
                    df.at[index, 'Language'] = result['language']
                    
            except Exception as e:
                print(f"Exception for row {index}: {e}")
    
    return df

In [46]:
# It is a time-consuming process (approximately 4 hours) 
# and doesn't provide comprehensive information for each book, 
# which is why I ultimately choose to skip this step.

# Call the function
# null_rows_df = update_books_df(null_rows_df)

In [47]:
# Calculate the percentage of data represented by null_values_df
percentage_to_drop = (len(null_rows_df) / len(books_df)) * 100

# Print the percentage
print(f"Percentage of data to drop: {percentage_to_drop:.2f}%")

Percentage of data to drop: 26.28%


In [48]:
# Drop rows from books_df where any column has null values
books_df.dropna(axis=0, how='any', inplace=True)

**The decision to remove rows with any null values from the "books_df" DataFrame is primarily driven by time constraints and the need to expedite the project's progress. While it is acknowledged that this action may lead to a reduction in the overall data quality and potentially affect the accuracy of the book recommender, it is a pragmatic choice made to maintain workflow efficiency. The time-consuming nature of making requests to various APIs has prompted this decision, as it allows for faster development and prevents undue delays in the project timeline.**

In [49]:
books_df.shape

(73719, 11)

In [50]:
books_df.isnull().sum()

author           0
description      0
genre            0
image_url        0
isbn             0
url              0
pages            0
rating           0
reviews          0
title            0
total_ratings    0
dtype: int64

### 2. users_books_df

I will fill the age missing values lately in the EDA part. And since I only have 1 missing value for author and 2 for publisher I will fill them manually checking in the web using the ISBN of the book.

(Finally I won't use this df so it's not in the EDA notebook).

In [51]:
# Identify rows with missing values in the 'author' and 'publisher' columns

missing_author_publisher = users_books_df[users_books_df['author'].isnull() | users_books_df['publisher'].isnull()]

In [52]:
missing_author_publisher

,location,age,isbn,rating,title,author,year,publisher,image_url
857163,"morrow, georgia, usa",52.0,193169656X,9,Tyrant Moon,Elaine Corvidae,2002,NaN,http://images.amazon.com/images/P/193169656X.0...
857174,"morrow, georgia, usa",52.0,1931696993,9,Finders Keepers,Linnea Sinclair,2001,NaN,http://images.amazon.com/images/P/1931696993.0...
857604,"selayang, selangor, malaysia",26.0,9627982032,8,The Credit Suisse Guide to Managing Your Perso...,NaN,1995,Edinburgh Financial Publishing,http://images.amazon.com/images/P/9627982032.0...


I manually searched for missing information on the web using the ISBN and filled in the missing data for specific ISBN values in the `users_books_df`.

- ISBN 193169656X: Filled 'publisher' with 'NovelBooks, Inc.'
- ISBN 1931696993: Filled 'publisher' with 'CreateSpace Independent Publishing Platform'
- ISBN 9627982032: Filled 'author' with 'n/a'

"By n/a (Author)" typically indicates that the author's name for a particular book or publication is not available or not provided. "n/a" stands for "not applicable" or "not available," and it is used as a placeholder when there is no specific author mentioned for the book. This can occur for various reasons, such as when the book is a compilation of works from multiple authors, when the author's name is unknown, or when the information is missing or not provided in the source data.

In [53]:
# Fill 'publisher' column for specific ISBN values
users_books_df.loc[users_books_df['isbn'] == '193169656X', 'publisher'] = 'NovelBooks, Inc.'
users_books_df.loc[users_books_df['isbn'] == '1931696993', 'publisher'] = 'CreateSpace Independent Publishing Platform'

# Fill 'author' column for a specific ISBN value
users_books_df.loc[users_books_df['isbn'] == '9627982032', 'author'] = 'n/a'

In [54]:
users_books_df.isnull().sum()

location          0
age          277835
isbn              0
rating            0
title             0
author            0
year              0
publisher         0
image_url         0
dtype: int64

### 3. gr_books_df

### Fill missing information using Google Books API

Same issue. As I previously said.

In [55]:
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import numpy as np

def fetch_book_data(isbn):
    url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'items' in data and data['totalItems'] > 0:
            book_info = data['items'][0]['volumeInfo']
            return book_info.get('description', ''), book_info.get('categories', [])
    return '', []

# Function to run fetch_book_data in parallel
def fetch_descriptions_and_genres(isbns):
    book_data = {}
    with ThreadPoolExecutor() as executor:
        future_to_isbn = {executor.submit(fetch_book_data, isbn): isbn for isbn in isbns}
        for future in as_completed(future_to_isbn):
            isbn = future_to_isbn[future]
            try:
                book_data[isbn] = future.result()
            except Exception as e:
                book_data[isbn] = ('', [])
    return book_data

# Function to update the DataFrame with descriptions and genres
def update_dataframe(df):
    isbns = df['isbn'].tolist()
    book_data = fetch_descriptions_and_genres(isbns)
    
    # Update DataFrame with the fetched data
    for isbn, (description, genres) in book_data.items():
        df.loc[df['isbn'] == isbn, 'Description'] = description
        df.loc[df['isbn'] == isbn, 'Genre'] = ', '.join(genres)  # Join genres list into a string
    
    return df

# Same issue. Takes too long.
# gr_books_df = update_dataframe(gr_books_df)

In [56]:
gr_books_df.shape

(11123, 10)

In [57]:
gr_books_df.isnull().sum()

title            0
author           0
rating           0
isbn             0
language         0
pages            0
ratings_count    0
reviews_count    0
year             0
publisher        0
dtype: int64

Finally, due time constrictions I will only use the books_df for clustering.

## Check Duplicates

In [58]:
print(books_df.duplicated().sum())
print(users_books_df.duplicated().sum())
print(best_selling_df.duplicated().sum())
print(goodreads_df.duplicated().sum())
print(gr_books_df.duplicated().sum())

0
3824
803
0
0


In [59]:
# Drop duplicates from the users_books_df
users_books_df = users_books_df.drop_duplicates()

users_books_df.duplicated().sum()

0

## Check % of null values, Data types and Unique Values

### 1. books_df

In [60]:
# % of null values
missing_percentage = (books_df.isnull().sum() / len(books_df)) * 100
missing_percentage

author           0.0
description      0.0
genre            0.0
image_url        0.0
isbn             0.0
url              0.0
pages            0.0
rating           0.0
reviews          0.0
title            0.0
total_ratings    0.0
dtype: float64

In [61]:
# Check data types
books_df.dtypes

author            object
description       object
genre             object
image_url         object
isbn              object
url               object
pages              int64
rating           float64
reviews            int64
title             object
total_ratings      int64
dtype: object

In [62]:
# See unique values in each column
books_df.nunique()

author           51979
description      73336
genre            61272
image_url        73719
isbn             73719
url              73719
pages             1248
rating             267
reviews           2734
title            72368
total_ratings     9716
dtype: int64

It seems that I have 73719 different isbn so **73.719 different books**.

### 2. users_books_df

In [63]:
missing_percentage_users_books = (users_books_df.isnull().sum() / len(users_books_df)) * 100
missing_percentage_users_books

location      0.000000
age          26.783587
isbn          0.000000
rating        0.000000
title         0.000000
author        0.000000
year          0.000000
publisher     0.000000
image_url     0.000000
dtype: float64

In [64]:
users_books_df.dtypes

location      object
age          float64
isbn          object
rating         int64
title         object
author        object
year          object
publisher     object
image_url     object
dtype: object

In [65]:
# Changing year to date type

unique_years = users_books_df['year'].unique()
unique_years

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1987, 1989, 1992, 1983, 1979, 1995, 1982, 1985, 1986,
       1978, 1980, 1952, 1990, 1981, 0, 1984, 1968, 1961, 1959, 1946,
       1974, 1958, 1976, 1971, '2002', 1977, 1975, 1965, 1941, 1970,
       '1996', '1999', 1972, '1989', '1992', '2001', 1953, 1962, 1973,
       '2004', 1960, '1997', '2000', '0', 1954, 1966, 1920, '1998',
       '2003', 1967, 1956, 1942, 1955, 1951, 1969, 1948, '1977', '1990',
       '1986', '1994', '1982', 1963, 1964, '1995', '1959', '1988', 1950,
       '1991', '1984', 2005, 1943, '1993', '1987', '1983', '1981', 1957,
       '1972', '1980', 1937, 1940, '1965', '1970', '1979', 1930, 1923,
       1945, 1947, '1976', 1936, '1985', '1978', 1925, '1966', '1958',
       '1975', '1973', 1927, '1974', '1967', '1971', '1969', 2030, '1968',
       2011, '1960', '1964', 2020, 1939, 1926, 1938, '1961', 1911, 1904,
       1949, 1932, 1929, '1953', '1963', '1955', '1944', '1920', '1

In [66]:
# Locate the values that contain letters or less than 3 digits
filtered_df = users_books_df[users_books_df['year'].str.contains('[A-Za-z]', na=False) | (users_books_df['year'].str.len() < 3)]

# Display the filtered DataFrame
filtered_df

,location,age,isbn,rating,title,author,year,publisher,image_url
82630,"bremen, bremen, germany",22.0,3453137205,7,Marchenmond,Wolfgang Hohlbein,0,Wilhelm Heyne Verlag GmbH &amp; Co KG,http://images.amazon.com/images/P/3453137205.0...
180775,"milan, lombardia, italy",21.0,8804382449,7,Fiori Nella Pioggia,Rosamunde Pilcher,0,Transglobal Books Inc,http://images.amazon.com/images/P/8804382449.0...
180776,"monza, lombardia, italy",26.0,8804382449,0,Fiori Nella Pioggia,Rosamunde Pilcher,0,Transglobal Books Inc,http://images.amazon.com/images/P/8804382449.0...
207865,"madrid, madrid, spain",31.0,840142285X,8,"Un Mundo Feliz (Fiction, Poetry and Drama)",Huxley,0,Plaza &amp; Janes Editor,http://images.amazon.com/images/P/840142285X.0...
213598,"roma, lazio, italy",29.0,8804401680,7,Ti Con Zero,Italo Calvino,0,Mondadori Italian,http://images.amazon.com/images/P/8804401680.0...
...,...,...,...,...,...,...,...,...,...
1030520,"omaha, nebraska, usa",NaN,0529044633,0,Charlie Brown Dictionary Volume 1,Charles Schulz,0,Riverside Dist@world Bible,http://images.amazon.com/images/P/0529044633.0...
1030533,"omaha, nebraska, usa",NaN,0553120166,0,Shroud,Robert K Wilcox,0,Bantam Doubleday Dell,http://images.amazon.com/images/P/0553120166.0...
1030756,"omaha, nebraska, usa",NaN,0716620073,0,Christmas In Britain,Edited,0,Random House~childrens,http://images.amazon.com/images/P/0716620073.0...
1030777,"omaha, nebraska, usa",NaN,0785307494,0,Card Games for One Or Two,David Galt,0,Publications International Ltd,http://images.amazon.com/images/P/0785307494.0...


In [67]:
filt_years = filtered_df['year'].unique()

In [68]:
filt_years

array(['0', 'DK Publishing Inc', 'Gallimard'], dtype=object)

In [69]:
users_books_df.shape

(1027312, 9)

In [70]:
percentage = (len(filtered_df) / len(users_books_df)) * 100
print(f"Percentage of values in filtered_df relative to users_books_df: {percentage:.2f}%")

Percentage of values in filtered_df relative to users_books_df: 0.12%


Given that the percentage of values in `filtered_df` relative to the total DataFrame `users_books_df` is only 0.12%, which is an extremely small and insignificant portion of the dataset, it is justifiable to proceed with the decision to remove these rows. Removing such a small fraction of the data will have negligible impact on the overall analysis and modeling while simplifying the dataset by eliminating rows with non-standard or irrelevant values. This action ensures that the dataset remains cleaner and more focused on the relevant data for the analysis.

In [71]:
# Define the values to be removed
values_to_remove = ['0', 'DK Publishing Inc', 'Gallimard']

# Remove rows with specified values in the 'year' column
users_books_df = users_books_df[~users_books_df['year'].isin(values_to_remove)]
users_books_df.shape

(1026044, 9)

In [72]:
unique_years = users_books_df['year'].unique()
unique_years

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1987, 1989, 1992, 1983, 1979, 1995, 1982, 1985, 1986,
       1978, 1980, 1952, 1990, 1981, 0, 1984, 1968, 1961, 1959, 1946,
       1974, 1958, 1976, 1971, '2002', 1977, 1975, 1965, 1941, 1970,
       '1996', '1999', 1972, '1989', '1992', '2001', 1953, 1962, 1973,
       '2004', 1960, '1997', '2000', 1954, 1966, 1920, '1998', '2003',
       1967, 1956, 1942, 1955, 1951, 1969, 1948, '1977', '1990', '1986',
       '1994', '1982', 1963, 1964, '1995', '1959', '1988', 1950, '1991',
       '1984', 2005, 1943, '1993', '1987', '1983', '1981', 1957, '1972',
       '1980', 1937, 1940, '1965', '1970', '1979', 1930, 1923, 1945, 1947,
       '1976', 1936, '1985', '1978', 1925, '1966', '1958', '1975', '1973',
       1927, '1974', '1967', '1971', '1969', 2030, '1968', 2011, '1960',
       '1964', 2020, 1939, 1926, 1938, '1961', 1911, 1904, 1949, 1932,
       1929, '1953', '1963', '1955', '1944', '1920', '1956',

In [73]:
# Convert the 'year' column to datetime
users_books_df['year'] = pd.to_datetime(users_books_df['year'], errors='coerce')

In [74]:
users_books_df.dtypes

location             object
age                 float64
isbn                 object
rating                int64
title                object
author               object
year         datetime64[ns]
publisher            object
image_url            object
dtype: object

In [75]:
users_books_df.nunique()

location      22464
age             141
isbn         269119
rating           11
title        240222
author       101246
year            196
publisher     16635
image_url    268814
dtype: int64

### 3. best_selling_df

In [76]:
missing_percentage_best_selling = (best_selling_df.isnull().sum() / len(best_selling_df)) * 100
missing_percentage_best_selling

title     0.0
price     0.0
rating    0.0
author    0.0
year      0.0
genre     0.0
url       0.0
dtype: float64

In [77]:
best_selling_df.dtypes

title      object
price     float64
rating    float64
author     object
year        int64
genre      object
url        object
dtype: object

In [78]:
unique_years = best_selling_df['year'].unique()
unique_years

array([2023, 2018, 2017, 2014, 2020, 1960, 1998, 2022, 2009, 2015, 2013,
       2011, 2021, 1967, 1997, 1969, 2019, 1947, 1989, 2016, 1980, 2024,
       2008, 2005, 1999, 1982, 1987])

In [79]:
# Convert the 'year' column to datetime
best_selling_df['year'] = pd.to_datetime(best_selling_df['year'], errors='coerce')

In [80]:
best_selling_df.dtypes

title             object
price            float64
rating           float64
author            object
year      datetime64[ns]
genre             object
url               object
dtype: object

In [81]:
best_selling_df.nunique()

title     86
price     83
rating     9
author    75
year      27
genre     70
url       87
dtype: int64

### 4. goodreads_df

In [82]:
missing_percentage_goodreads = (goodreads_df.isnull().sum() / len(goodreads_df)) * 100
missing_percentage_goodreads

title        0.0
author       0.0
image_url    0.0
rating       0.0
dtype: float64

In [83]:
goodreads_df.dtypes

title        object
author       object
image_url    object
rating       object
dtype: object

In [84]:
goodreads_df.nunique()

title        100
author        91
image_url    100
rating       100
dtype: int64

In [85]:
goodreads_df.head()

,title,author,image_url,rating
0,"Fourth Wing (The Empyrean, #1)",Rebecca Yarros,https://i.gr-assets.com/images/S/compressed.ph...,"4.63 avg rating — 844,282 ratings"
1,Happy Place,Emily Henry,https://i.gr-assets.com/images/S/compressed.ph...,"4.06 avg rating — 574,386 ratings"
2,Yellowface,R.F. Kuang,https://i.gr-assets.com/images/S/compressed.ph...,"3.87 avg rating — 227,094 ratings"
3,"Love, Theoretically",Ali Hazelwood,https://i.gr-assets.com/images/S/compressed.ph...,"4.17 avg rating — 239,576 ratings"
4,"Divine Rivals (Letters of Enchantment, #1)",Rebecca Ross,https://i.gr-assets.com/images/S/compressed.ph...,"4.26 avg rating — 161,237 ratings"


I will split the "rating" column into two separate columns "rating" and "rating_count".

In [86]:
# Split the "rating" column into "rating" and "rating_count" columns
goodreads_df[['rating', 'rating_count']] = goodreads_df['rating'].str.extract(r'([\d.]+) avg rating — ([\d,]+) ratings')

In [87]:
goodreads_df.head()

,title,author,image_url,rating,rating_count
0,"Fourth Wing (The Empyrean, #1)",Rebecca Yarros,https://i.gr-assets.com/images/S/compressed.ph...,4.63,"844,282"
1,Happy Place,Emily Henry,https://i.gr-assets.com/images/S/compressed.ph...,4.06,"574,386"
2,Yellowface,R.F. Kuang,https://i.gr-assets.com/images/S/compressed.ph...,3.87,"227,094"
3,"Love, Theoretically",Ali Hazelwood,https://i.gr-assets.com/images/S/compressed.ph...,4.17,"239,576"
4,"Divine Rivals (Letters of Enchantment, #1)",Rebecca Ross,https://i.gr-assets.com/images/S/compressed.ph...,4.26,"161,237"


### 5. gr_books_df

In [88]:
missing_percentage_gr_books = (gr_books_df.isnull().sum() / len(gr_books_df)) * 100
missing_percentage_gr_books

title            0.0
author           0.0
rating           0.0
isbn             0.0
language         0.0
pages            0.0
ratings_count    0.0
reviews_count    0.0
year             0.0
publisher        0.0
dtype: float64

In [89]:
gr_books_df.dtypes

title             object
author            object
rating           float64
isbn              object
language          object
pages              int64
ratings_count      int64
reviews_count      int64
year              object
publisher         object
dtype: object

In [90]:
gr_books_df.nunique()

title            10348
author            6639
rating             209
isbn             11123
language            27
pages              997
ratings_count     5294
reviews_count     1822
year              3679
publisher         2290
dtype: int64

Due time constrictions I will finally not use this df.

## Saving

In [91]:
# Drop "isbn" before saving
books_df.drop(columns=['isbn'], inplace=True)
# Save books_df to CSV
books_df.to_csv('books_df_cleaned.csv', index=False)

# Save users_books_df to CSV
users_books_df.to_csv('users_books_df_cleaned.csv', index=False)

# Save best_selling_df to CSV
best_selling_df.to_csv('best_selling_df_cleaned.csv', index=False)

# Save goodreads_df to CSV
goodreads_df.to_csv('goodreads_df_cleaned.csv', index=False)

# Save gr_books_df to CSV
gr_books_df.to_csv('gr_books_df_cleaned.csv', index=False)